In [1]:
import numpy as np
from src.data.preprocess.dataturks.preprocess_translearn_hatespeech import create_NN_sets
from src.evaluation.hatespeech.evaluation_translearn_hatespeech import evaluate_best_model
from src.models.hatespeech.model_translearn_hatespeech import fine_tune_model

np.random.seed(42)
vocab_size = 10000
path_to_target_data = "../../data/external/hatespeech/clean_sexism_dataset.csv"
path_to_original_model = "../../models/example_dataturks.h5"
path_to_fine_tuned_model = "../../models/fine_tuned.h5"
path_to_fine_tuned_model2 = "../../models/fine_tuned_last_layer.h5"
path_to_sexist_model = "../../models/zeerak_model.h5"

datasets = create_NN_sets(path_to_target_data, vocab_size)
# fine_tune_model(path_to_original_model, path_to_fine_tuned_model, datasets)
print("\n Evaluation of model with ALL layers fine-tuned:")
evaluate_best_model(path_to_fine_tuned_model, datasets[4], datasets[5], datasets[6], 10000)

# fine_tune_model(path_to_original_model, path_to_fine_tuned_model2, datasets)
print("\n Evaluation of model with LAST layer fine-tuned:")
evaluate_best_model(path_to_fine_tuned_model2, datasets[4], datasets[5], datasets[6], 10000)

# train_model(path_to_sexist_model, datasets, vocab_size)
print("\n Evaluation of model trained from scratch:")
evaluate_best_model(path_to_sexist_model, datasets[4], datasets[5], datasets[6], 10000)


Using TensorFlow backend.


Words in index: 22101

 Evaluation of model with ALL layers fine-tuned:
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2982/2982 [==============================] - 0s 60us/step
acc: 87.42%, loss: 0.34
Confusion matrix:
[[2181  139]
 [ 236  426]]
Marco F1:0.807606
Micro F1:0.874245
Weighted F1:0.870562
11924    1
11925    0
11926    0
11927    0
11928    0
11929    0
11930    0
11931    0
11932    0
11933    0
11934    0
11935    1
11936    0
11937    1
11938    0
11939    0
11940    0
11941    0
11942    0
11943    0
11944    1
11945    0
11946    0
11947    0
11948    0
11949    0
11950    0
11951    0
11952    0
11953    0
        ..
14876    1
14877    0
14878    1
14879    0
14880    0
14881    0
14882    1
14883    0
14884    0
14885    0
14886    1
14887    1
14888    0
14889    0
14890    0
14891    0
14892    0
14893    0
14894    0
14895  